In [12]:
system("sudo apt-get update -y && sudo apt-get install -y jags && sudo apt-get install -y r-cran-rjags")
install.packages('runjags')
install.packages('rjags')

In [13]:
# Charger les bibliothèques nécessaires
library(dplyr)
library(ggplot2)
library(tidyverse)
library(runjags)
library(rjags)

# Introduction
L’étude des disparités salariales est un sujet central en économie du travail et en sciences sociales. Malgré les avancées vers l’égalité des chances, des différences de salaires persistent entre divers groupes sociaux, notamment en fonction du genre, de l’ethnicité, du niveau d’éducation et de l’expérience professionnelle. Aux États-Unis, des études ont montré que les écarts salariaux sont significatifs : les femmes gagnent en moyenne moins que les hommes, et certaines minorités ethniques, comme les Afro-Américains et les Latino-Américains, perçoivent des salaires inférieurs à ceux des Américains blancs et asiatiques.

Notre projet vise à identifier les principaux facteurs influençant les salaires horaires en utilisant des modèles bayésiens. Plus précisément, nous cherchons à quantifier l’impact de variables telles que le genre, l’expérience professionnelle, le niveau d’éducation, l’appartenance syndicale et la localisation géographique sur les salaires.

Pour ce faire, nous utiliserons des méthodes bayésiennes de régression mises en œuvre avec JAGS, permettant une estimation robuste et une meilleure quantification des incertitudes. Notre approche inclura l’exploration des données, la spécification des modèles avec des choix de lois a priori justifiées, ainsi que des évaluations rigoureuses des performances des modèles à travers des diagnostics de convergence et des simulations de données fictives (fake data check).

Ce rapport est structuré comme suit : après une présentation des données et de leur exploration, nous décrirons les modèles bayésiens retenus et justifierons les choix de lois a priori. Nous présenterons ensuite les résultats de l’ajustement des modèles, suivis des diagnostics de convergence et de l’évaluation de leur performance. Enfin, nous discuterons des conclusions et des implications de nos résultats dans le contexte des disparités salariales aux États-Unis.

# Modèle génératif et espace des paramètres

On veut construire un modèle prédictif pour le salaire des individus pour lesquels on connait 10 variables explicatives (âge, éducation, experience professionnelle, etc.), 434 individus dans un jeu de données d'entraînement et 100 individus dans le jeu de données de test.

On note $Y_i \in \mathbb{R}$ le salaire de l'individu $i$ et $X_i \in \mathbb{R}^{10}$ les variables explicatives de l'individu $i$.

Régression linéaire:
$Y_i = \beta_0 + \beta_1 X_{1i} + \beta_2 X_{2i} + \cdots + \beta_{10} X_{10i} + \sigma \epsilon_i$
11 paramètres $\beta = (\beta_0, \beta_1, \beta_2, \cdots, \beta_{10})$  $\epsilon_i \sim \mathcal{N}(0, 1)$ et $\sigma \in \mathbb{R^+}$

Certaines variables peuvent avoir peu ou pas d'effet sur le salaire, ie des $\beta_i = 0$.

## Distribution à postériori

A partir de $y=(y_1, y_2, \cdots, y_{434})$ et $ X=(X_1, X_2, \cdots, X_{434})$, on peut calculer $P(\beta | y, X)$ et regarder $P(\beta_j \neq 0 | y, X)$ pour chaque coefficient.

## Performances prédictives

Un manière d'évaluer la performance prédictive $\hat{\beta}_{bayes} = E(\beta | y, X)$
$X_{test}$ matrice 100x10
$y_{test}$ les observations
$y_{test,Bayes} = X_{test} \hat{\beta}_{bayes}$

# Degrès de croyance

Que croit-on sur l'effet de chaque variable explicative sur le salaire?

Nous pouvons formuler des hypothèse à priori sur l'impact des différentes variables :

- Éducation : On s’attend à une relation positive entre le niveau d’éducation et le salaire.

- Expérience professionnelle (workexp) : Plus d’expérience est généralement associée à un salaire plus élevé.

- Sexe (female) : Il est bien documenté qu’un écart salarial entre hommes et femmes existe (gender pay gap).

- Région (south) : Il peut y avoir des différences salariales régionales.

- Appartenance syndicale (unionmember) : Être syndiqué peut influencer le salaire.

- Âge (age) : En général, l’âge est corrélé à l’expérience professionnelle, mais il pourrait aussi capturer d’autres dynamiques comme la discrimination liée à l’âge.

- Ethnicité (ethnicity) : Certaines inégalités salariales peuvent exister selon l’origine ethnique.

- Occupation et secteur (occupation, sector) : Le type d’emploi et le secteur d’activité influencent fortement le salaire.

- Statut matrimonial (married) : Certaines études suggèrent que les personnes mariées ont des salaires plus élevés, en particulier les hommes.


### Estimation d'un paramètre $\beta$ avec des données

Ayant obtenu {$Y=y$} , $P(\beta | y,X) = \frac{P(\beta) P(y,X | \beta)}{P(y,X)}$

Donc $posterior \propto prior \times likelihood$

$prior = P(\beta)$ : Croyance des effets des variables sur le salaire avant d'avoir les données.
$likelihood = P(y | \beta)$ : Probabilité d'observer les données en fonction des paramètres du modèle
$posterior = P(\beta | y)$ : La Distribution à posteriori après avoir observé les données*

On peut le voir ainsi :

$P(Model | New Data)  = \frac{P(Model) P(New Data | Model)}{P(New Data)}$



# Nos données

Notre jeu de données comprend 534 individus. Chaque individu a 10 variables : education, south, female, workexp, unionmember, wages, age, ethnicity, occupation, sector, married.

- Variables quantitatives:
    > **education**: Années d'études
    >**workexp**: Années d'expêrience professionnelle
    >**age**: âge

- Variables qualitatives:
    >**south**: Vit dans le sud ou non
    >**female**: Indicateur de genre (0 = Homme, 1 = Femme)
    >**unionmember**: Indicateur de membre du syndicat (0 = Non, 1 = Oui)
    >**married**: Indicateur de statut marital. Marié(e) ou non.

- Variables categorielles:
    >**ethnicity**: Ethnicité (White, Hispanic, Other)
    >**occupation**: Occupation (Management, Sales, Clerical, Professional, Service, Other)
    >**sector**: Secteur d'activité (Manufactoring, Construction, Other)




In [3]:
wage_data = read.csv("https://raw.githubusercontent.com/derghalmanal/Wage/refs/heads/main/wage_data.csv", header=TRUE, sep=",")
head(wage_data, 5)

In [4]:
str(wage_data)

Analysons la contribution de chaque variable aux salaires

In [15]:
plot_credible_intervals <- function(fit) {
  # Extract the MCMC samples and the names of the parameters
  samples <- as.data.frame(jags_samples$mcmc %>% lapply(as_tibble) %>% bind_rows())
  params <- names(samples)

  # Calculate the 50% and 95% credible intervals for each parameter
  intervals <- data.frame(param = character(), lower = numeric(), upper = numeric())
  for (param in params) {
    est_mean <- mean(samples[[param]])
    est_median <- median(samples[[param]])
    ci_50_infCI <- quantile(samples[[param]], probs = 0.25)
    ci_50_supCI <- quantile(samples[[param]], probs = 0.75)
    ci_95_infCI <- quantile(samples[[param]], probs = 0.025)
    ci_95_supCI <- quantile(samples[[param]], probs = 0.975)
    intervals <- rbind(intervals, data.frame(param = param, est_mean = est_mean, est_median = est_median, ci_50_infCI = ci_50_infCI, ci_50_supCI = ci_50_supCI, ci_95_infCI = ci_95_infCI, ci_95_supCI = ci_95_supCI))
  }

  # Create a ggplot object
  p <- ggplot(intervals, aes(y = param)) +
    theme_classic() +
    geom_segment(aes(y = param, yend = param, x = ci_95_infCI, xend = ci_95_supCI),
      color = "red", size = 0.5
    ) +
    geom_segment(aes(y = param, yend = param, x = ci_50_infCI, xend = ci_50_supCI),
      color = "red", size = 1.5
    ) +
    geom_point(aes(x = est_mean), size = 3) +
    labs(title = "Posterior credible intervals") +
    xlab("") +
    ylab("")

  # Print the plot
  print(p)
}

# Variable Education

La variable Education représente le nombre d'années d'études.

In [5]:
# Charger les bibliothèques nécessaires
library(ggplot2)

# Tracer l'histogramme avec la densité empirique
ggplot(wage_data, aes(x = education)) +
  geom_histogram(aes(y = ..density..), bins = 30, fill = "skyblue", color = "black", alpha = 0.7) +
  labs(title = "Distribution de la variable 'education' avec densité Gamma(25,2)",
       x = "Niveau d'éducation", y = "Densité") +
  theme_minimal()


L'histogramme montre une distribution avec un pic très prononcé à 12 années d'études, indiquant que la plupart des individus ont ce niveau d'éducation.
Il y a quelques valeurs dispersées autour de ce pic, mais la majorité des données est concentrée à 12 ans.
Choix de la vraisemblance :

Étant donné la concentration des données à une valeur spécifique, nous avons choisi une distribution de Poisson pour modéliser le nombre d'années d'études. 

$y,X | \beta_{education} \sim \mathcal{Poisson}(\beta_{education} )$

## Prior

$\beta_{education} \sim \mathcal{Gamma}(\alpha, \beta)$

On souhaite que $E(\beta_{education}) = 12$ et $V(\beta_{education}) = 20$
Pour la loi gamma $E(\beta) = \frac{\alpha}{\beta}$ et $V(\beta) = \frac{\alpha}{\beta^2}$

donc $\beta_{education} \sim \mathcal{Gamma}(7.2, 0.6)$

## Implementation en JAGS

In [24]:
model_education_string <- "
model {
  beta_education ~ dgamma(7.2, 0.6)
  
  
  for (i in 1:n) {
    y[i] ~ dpois(beta_education)
  }
  
}
"


# Définition des données
y <- wage_data$wages
data_list <- list(y = y, n = length(y))


# Compiling and producing posterior samples from the model.
jags_samples <- run.jags(model = model_education_string, data = data_list, monitor = c("beta_education"))

# Plotting and summarizing the posterior distribution
jags_samples
plot_credible_intervals(jags_samples)

## MCMC

In [40]:
# Fonction pour tracer traceplots et autocorrélogrammes
plot_mcmc <- function(jags_samples) {
  
  # Convertir en objet mcmc
  mcmc_samples <- as.mcmc.list(jags_samples)
  
  # Traceplots (convergence)
  plot(mcmc_samples, trace = TRUE, density = TRUE)
  
  # Tracer l'autocorrélation pour chaque chaîne
  autocorr.plot(jags_samples)
}

# Utilisation de la fonction
plot_mcmc(jags_samples)



## Fake data check

In [29]:
generate_fake_data <- function(n) {
    beta_education_fake <- 9
  
  # Générer des données factices à partir de la distribution de Poisson
  y_fake <- rpois(n, beta_education_fake)
  
  return(list(y_fake = y_fake))
}

# Générer des données factices à partir des échantillons postérieurs
fake_data <- generate_fake_data(length(y))

# Comparer les données factices avec les données réelles
summary(fake_data$y_fake)
summary(y)

data_list <- list(y = fake_data$y_fake, n = length(y))


# Compiling and producing posterior samples from the model.
jags_samples <- run.jags(model = model_education_string, data = data_list, monitor = c("beta_education"))

# Plotting and summarizing the posterior distribution
jags_samples
plot_credible_intervals(jags_samples)

## Conclusion

# Variable Age

On observe que l'age des individus est fortement correle avec l'experience professionnelle, ce qui fait sens puisque plus l'age est grand, plus l'experience professionnelle est longue. Pour éviter une instabilité des estimations, nous supprimons écartons la variable Age.

In [42]:
ggplot(wage_data, aes(x=age, y=workexp)) +
  geom_point(alpha=0.5, color="purple") +
  labs(title="Relation entre Age et Expérience Professionnelle", x="Age", y="Expérieence Professionnelle")

# Variable Work experience

In [56]:
# Tracer l'histogramme avec la densité empirique
ggplot(wage_data, aes(x = workexp)) +
  geom_histogram(aes(y = ..density..), bins = 20, fill = "skyblue", color = "black", alpha = 0.7) +
  labs(title = "Distribution de la variable 'workexp'",
       x = "Experience professionnelle", y = "Densité") +
  theme_minimal()

## Choix du prior

Choix de la vraisemblance :

Une distribution normale semble appropriée pour modéliser les années d'expérience au travail, car les données suivent une forme de cloche.

$y,X | \beta_{workexp} \sim \mathcal{Normal}(\mu, \sigma)$

Choix du prior :

$\mu \sim \mathcal{Uniform}(0, 100)$ Si on ne sait pas précisément où se trouve la moyenne d'âge

$\sigma \sim \mathcal{Uniform}(0, 100)$ Si on ne sait pas preciser la variance

## Implémentation en JAGS

In [61]:
model_workexp_string <- "
model {

  mu ~ dunif(0, 30)
  sigma ~ dunif(0, 50)
  precision <- 1/sigma^2
  
  for (i in 1:n) {
    y[i] ~ dnorm(mu, precision)
  }
  
}
"


# Définition des données
y <- wage_data$workexp
data_list <- list(y = y, n = length(y))


# Compiling and producing posterior samples from the model.
jags_samples <- run.jags(model = model_workexp_string, data = data_list, monitor = c("mu", "sigma"))

# Plotting and summarizing the posterior distribution
jags_samples
plot_credible_intervals(jags_samples)

## MCMC

In [62]:
plot_mcmc(jags_samples)

## Fake data check

In [65]:
generate_fake_data <- function(n) {
    mu_fake <- 17
    sigma_fake <- 11
  
  # Générer des données factices à partir de la distribution de Poisson
  y_fake <- dnorm(n, mu_fake, sigma_fake)
  
  return(list(y_fake = y_fake))
}

# Générer des données factices à partir des échantillons postérieurs
fake_data <- generate_fake_data(length(y))

# Comparer les données factices avec les données réelles
summary(fake_data$y_fake)
summary(y)

data_list <- list(y = fake_data$y_fake, n = length(y))


# Compiling and producing posterior samples from the model.
jags_samples <- run.jags(model = model_education_string, data = data_list, monitor = c("mu_fake", "sigma_fake"))

# Plotting and summarizing the posterior distribution
jags_samples
plot_credible_intervals(jags_samples)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=650b0311-43bb-4704-b320-0c62dbd2dedc' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>